In [1]:
#Import packages
import numpy as np

In [20]:
#Simulation of preference table
def generatePreferenceTable(n):
    '''
    Given an even number n, return a dictionary of list of length (n-1). Each list represents the preference list of a person.
    The list of person k is randomly generated permutation of ndarray excluding k.
    
    Input:
    n: An even number.
    
    Output:
    dic: dictionary with key values 1,...,n
    '''
    dic = dict();
    
    for i in np.arange(1,n+1):
        toBePermuted = np.arange(1,n+1);
        dic[i] = np.random.permutation(np.delete(toBePermuted, np.where(toBePermuted == i)));
    
    return dic;

#Pretty Print function
def printPreferenceTable(table):
    '''
    Give a preference table represented by a dictionary of ndarray, print it line by line.
    
    Input:
    table:A dictionary of ndarray.
    '''
    
    for i in np.arange(1, len(table)+1):
        print(f'{i}  {table[i]}');
    
    

In [47]:
#Example
dic = generatePreferenceTable(4)
printPreferenceTable(dic)

1  [4 2 3]
2  [4 3 1]
3  [4 1 2]
4  [2 3 1]


In [37]:
#Helper functions

def deletePair(table,i,j):
    '''
    Delete the pair {i,j} from the table.
    '''

    table[i] = np.delete(table[i], np.where(table[i] == j));
    table[j] = np.delete(table[j], np.where(table[j] == i));
    
def deleteLessPreferable(table, x, y):
    '''
    Delete any pair {x',y} where y prefers x than x'
    '''
    #First get y's preference list
    yList = table[y];
    #Get x's position in y's preference list
    xPos = (np.where(yList == x))[0];
    
    #Delete pairs after xPos
    for i in np.arange(xPos + 1, len(yList)):
        deletePair(table, yList[i], y);

In [45]:
#Phase I function
def srmPhaseI(table):
    '''
    Given a preference table represented by a dictionary of ndarray, executing Phase I of Stable Roommate Maching.
    
    Input:
    table: A dictionary of list
           table[k] is the preference list of person k.
           The preference list needs to be complete.
           
    Output:
    Raise exception if no stable matching exists.
    '''
    
    #Number of persons
    n = len(table);
    
    #semiengagement dictionary
    #semiengagementDic[y] = x if y accepts x's proposal, in this case, we say x is semiengaged with y
    semiengagementDic = dict();
    
    #Assign each person to be free
    freeList = list(np.arange(1, n+1));
    
    while len(freeList) != 0:
        x = freeList.pop(); #Let x be any free person
        
        
        #If one of the free person has empty list
        if len(table[x]) == 0:
            raise Exception("Empty list detected during Phase I. No stable matching.")
            
        y = table[x][0]; #Let y be the first person on x's preference list
        
        #If z is semiengaged to y
        if y in semiengagementDic:
            #Then let z be free
            freeList.append(semiengagementDic[y]);
        
        #Let x be semiengaged to y 
        semiengagementDic[y] = x;
        
        #Delete any pair {x',y} such that y prefers x than x'
        deleteLessPreferable(table, x, y);
        

In [96]:
dic = generatePreferenceTable(8)
printPreferenceTable(dic)

srmPhaseI(dic);
printPreferenceTable(dic);

1  [4 6 2 8 3 7 5]
2  [4 8 6 3 5 1 7]
3  [6 1 8 4 5 7 2]
4  [2 3 8 6 7 5 1]
5  [7 8 2 1 6 4 3]
6  [5 2 4 3 8 1 7]
7  [6 2 1 4 3 8 5]
8  [2 4 3 1 6 7 5]
1  [8 3 7]
2  [4]
3  [6 1 8]
4  [2]
5  [7 6]
6  [5 3]
7  [1 5]
8  [3 1]


In [210]:
#Helper functions
def findFirstNonsingle(table):
    '''
    Given a table, return the first person with nonsingle list.
    If no such person exists, return 0.
    '''
    for i in np.arange(1, len(table)+1):
        if len(table[i]) > 1:
            return i;
    
    return 0;

def nextPersonIntheRotation(table,i):
    '''
    Given a table and a person whose list is not single, find the next person in the Phase II SRM.
    '''
    
    y = table[i][1];
    
    return table[y][-1]

def eliminateRotation(table, rotation):
    '''
    Given a table and a rotation, eliminate rotation from table.
    '''
    
    for i in range(1,len(rotation)):
        x_i_minus_one = rotation[i-1][0];
        yi = rotation[i][1];
        if x_i_minus_one not in table[yi]:
            raise Exception("Empty list detected during Phase II. No stable matching.")
        deleteLessPreferable(table, x_i_minus_one, yi);
    
    
    if rotation[-1][0] not in table[rotation[0][1]]:
        raise Exception("Empty list detected during Phase II. No stable matching.")
    deleteLessPreferable(table, rotation[-1][0], rotation[0][1]);
    

In [105]:
dic = generatePreferenceTable(8)
printPreferenceTable(dic)

srmPhaseI(dic);
printPreferenceTable(dic);

findFirstNonsingle(dic)

1  [5 7 3 8 4 2 6]
2  [7 1 4 6 8 3 5]
3  [4 8 2 6 5 7 1]
4  [7 5 6 2 3 1 8]
5  [1 3 4 8 7 6 2]
6  [3 5 8 7 1 4 2]
7  [8 3 5 1 6 2 4]
8  [3 1 4 5 6 7 2]
1  [5]
2  [7 4]
3  [4 8]
4  [2 3]
5  [1]
6  [8 7]
7  [6 2]
8  [3 6]


2

In [113]:
nextPersonIntheRotation(dic, 6)

2

In [182]:
#Phase II function
def srmPhaseII(table):
    '''
    Given a phase I preference table, either finish Phase II or raise the exception that no stable matching exists.
    '''
    
    #Find the number of people
    n = len(table);
    
    #Let x be the first person with nonsingle list
    x = findFirstNonsingle(table);
    
    #If no such x exists, i.e., every person's list only contain one person, we find the stable matching.
    if x == 0:
        return
    
    #isEmptyList = False;
    stack = list();
    
    while x <= n:
        
        #If the stack is emtpy
        if len(stack) == 0:
            while x <= n and len(table[x]) == 1:
                x = x + 1;
            if x <= n:
                stack.append(x);
        #If the stack is not empty       
        else:
            p = stack[-1]; #Let p be the person on top of the stack
            p = nextPersonIntheRotation(table, p)
            
            while p not in stack:
                stack.append(p);
                p = nextPersonIntheRotation(table, p);
            
            pIter = stack.pop();
            
            rotation = list();
            rotation.append((pIter, table[pIter][0]));
            
            while pIter != p:
                pIter = stack.pop();
                rotation.insert(0, (pIter, table[pIter][0]));
            
            eliminateRotation(table,rotation)
                
    
    
    

In [215]:
dic = generatePreferenceTable(8)
printPreferenceTable(dic)

srmPhaseI(dic)
printPreferenceTable(dic)

1  [4 2 5 6 8 3 7]
2  [6 7 8 5 1 4 3]
3  [7 4 2 5 1 6 8]
4  [3 8 6 2 1 5 7]
5  [3 7 6 4 1 2 8]
6  [3 1 4 7 2 8 5]
7  [1 3 8 4 6 5 2]
8  [3 5 2 4 1 6 7]
1  [2 5]
2  [6 8 5 1]
3  [7]
4  [8 6]
5  [1 2 8]
6  [4 2]
7  [3]
8  [5 2 4]


In [216]:
srmPhaseII(dic);
printPreferenceTable(dic);

1  [5]
2  [8]
3  [7]
4  [6]
5  [1]
6  [4]
7  [3]
8  [2]
